In [2]:
from CIRESA import maven, stereo_a, omni, solo, psp, CIR_detector, plot_heliosphere
import pandas as pd

# Define the start and end dates
start_date = '2022-08'
#start_date = '2019-11'
end_date = '2022-09'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

df = solo.load(months)

#CIR = CIR_detector.plot_and_choose_CIR(df)

C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-08.parquet


In [3]:

from CIRESA import maven, stereo_a, omni, solo, psp, CIR_detector, plot_heliosphere
import pandas as pd
# Define the start and end dates
start_date = '2022-07'
#start_date = '2019-11'
end_date = '2022-10'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

#months = '2022-10'

df = solo.load(months)
df4 = omni.load(months)
df2 = psp.load(months)
df3 = stereo_a.load(months)
df5 = maven.load(months, reduce_cadence='3H', interpolate=True)

df5

C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-08.parquet
C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-09.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-08.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-09.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-08.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-09.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-08.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-09.parquet
C:/Users/14milosi/CIRESA/reduced_data/maven\maven_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/maven\maven_data2022-08.parquet
C:/Users/14milosi/CIRESA/r

,B_R,B_T,B_N,B,X,Y,Z,V_R,V_T,V_N,...,LAT,INERT_LON,R,P_t,P_B,P,Beta,S_P,POL,Spacecraft_ID
2022-07-01 00:00:00,4.105,-10.01750,2.28000,14.409264,2770.0,6020.000000,-1720.0,285.000000,-48.799999,123.000000,...,5.532922,-93.562850,1.381952,36.338575,82.612117,107.317434,0.43987,64.655054,1.0,7.0
2022-07-01 00:06:00,4.329,-10.21525,2.14625,14.722511,2768.0,6019.666667,-1660.4,285.066667,-48.462849,121.180000,...,5.532986,-93.560152,1.381953,36.318515,87.375232,112.229027,0.43049,64.264405,1.0,7.0
2022-07-01 00:12:00,4.553,-10.41300,2.01250,15.035758,2766.0,6019.333333,-1600.8,285.133333,-48.125699,119.360000,...,NaN,NaN,1.381955,36.298456,92.138347,117.140620,0.42111,63.873756,1.0,7.0
2022-07-01 00:18:00,4.777,-10.61075,1.87875,15.349004,2764.0,6019.000000,-1541.2,285.200000,-47.788549,117.540000,...,NaN,NaN,1.381956,36.278396,96.901462,122.052213,0.41173,63.483107,1.0,7.0
2022-07-01 00:24:00,5.001,-10.80850,1.74500,15.662251,2762.0,6018.666667,-1481.6,285.266667,-47.451399,115.720000,...,NaN,NaN,1.381957,36.258337,101.664577,126.963806,0.40235,63.092457,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30 22:12:00,3.530,-2.17000,-3.44000,14.866515,3140.0,-3060.000000,6710.0,570.000000,-28.600000,71.099998,...,4.040569,-37.169737,1.447363,122.343404,87.938384,136.440321,1.39124,84.732724,1.0,7.0
2022-09-30 22:18:00,3.530,-2.17000,-3.44000,14.866515,3140.0,-3060.000000,6710.0,570.000000,-28.600000,71.099998,...,4.040569,-37.169737,1.447363,122.343404,87.938384,136.440321,1.39124,84.732724,1.0,7.0
2022-09-30 22:24:00,3.530,-2.17000,-3.44000,14.866515,3140.0,-3060.000000,6710.0,570.000000,-28.600000,71.099998,...,4.040569,-37.169737,1.447363,122.343404,87.938384,136.440321,1.39124,84.732724,1.0,7.0
2022-09-30 22:30:00,3.530,-2.17000,-3.44000,14.866515,3140.0,-3060.000000,6710.0,570.000000,-28.600000,71.099998,...,4.040569,-37.169737,1.447363,122.343404,87.938384,136.440321,1.39124,84.732724,1.0,7.0


In [ ]:
#PLOT MOVIE

from CIRESA import maven, stereo_a, omni, solo, psp, CIR_detector, plot_heliosphere
import pandas as pd
# Define the start and end dates
start_date = '2022-07'
#start_date = '2019-11'
end_date = '2022-10'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

#months = '2022-10'

df = solo.load(months)
df4 = omni.load(months)
df2 = psp.load(months)
df3 = stereo_a.load(months)
df5 = maven.load(months, reduce_cadence='3H', interpolate=True)


plot_heliosphere.progressive_model_movie([df, df3, df4]
                                         , plot_cadence = 24, persistance= 5
                                         , directory= 'Debugging'
                                         , sim_resolution=2
                                         , HEE = True
                                         , back_prop=False
                                         , movie= True
                                         , rlim = 1.5
                                         , COR=0
                                         , virtual_spacecraft=1
                                         , model='inelastic'
                                         )


C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-08.parquet
C:/Users/14milosi/CIRESA/reduced_data/solo\solo_data2022-09.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-08.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-09.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-08.parquet
C:/Users/14milosi/CIRESA/reduced_data/psp\psp_data2022-09.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-08.parquet
C:/Users/14milosi/CIRESA/reduced_data/stereo_a\stereo_a_data2022-09.parquet
C:/Users/14milosi/CIRESA/reduced_data/maven\maven_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/maven\maven_data2022-08.parquet
C:/Users/14milosi/CIRESA/r

In [ ]:
#PLOT MOVIE

from CIRESA import maven, stereo_a, omni, propagation, plot_heliosphere
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

matplotlib.use('TkAgg')

# Define the start and end dates
start_date = '2022-07'
#start_date = '2019-11'
end_date = '2022-09'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()


df4 = omni.load(months)


prop = propagation.inelastic_radial(df4['2022-07-01': '2022-07-20'])

plot_heliosphere.plot_spacecraft_carrington(prop, rlim=3)
plt.show()

C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-07.parquet
C:/Users/14milosi/CIRESA/reduced_data/omni\omni_data2022-08.parquet


In [19]:
prop['CARR_LON_RAD']# = prop['CARR_LON_RAD']*180/np.pi
#(prop['CARR_LON']%360)/180*np.pi
#prop['CARR_LON_RAD'] = (prop['CARR_LON']%360)/180*np.pi
#prop['CARR_LON_RAD']

2022-06-01 00:00:00    5.379867
2022-06-01 09:42:00    5.286747
2022-06-01 09:42:00    5.286518
2022-06-01 19:24:00    5.193627
2022-06-01 19:24:00    5.193398
                         ...   
2022-06-30 21:48:00    4.754854
2022-06-30 21:48:00    4.754603
2022-06-30 21:48:00    4.754353
2022-06-30 21:48:00    4.754104
2022-06-30 21:48:00    4.786607
Name: CARR_LON_RAD, Length: 2850, dtype: float64

In [ ]:
from CIRESA import stereo_a, omni, solo, psp, maven

import pandas as pd

# Define the start and end dates
start_date = '2024-01'
#start_date = '2019-11'
end_date = '2024-09'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

#maven.download_reduce_save_space(month=months, cadence='0.1H')
#omni.download_reduce_save_space(month=months, cadence='0.1H')
#stereo_a.download_reduce_save_space(month=months, cadence='0.1H')
#solo.download_reduce_save_space(month=months, cadence='0.1H')
psp.download_reduce_save_space(month=months, cadence='0.1H')



In [ ]:
from CIRESA import maven
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')

df = maven.load(['2022-09'], reduce_cadence='6H', interpolate=True)
df['Spacecraft_ID'] = 7
print(df)
#maven.plot(df)
#plt.show()

In [ ]:
from CIRESA import stereo_a, omni, solo, psp, maven
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define the start and end dates
start_date = '2024-09'
#start_date = '2019-11'
end_date = '2024-09'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

for month in months:
    df = solo.load(month=month)

    df['P_t'] = (df['N'] * df['V']**2) / 10**19 / 1.6727   * 10**6 *10**9 # J/cm^3 to nPa
    df['P_B'] = df['B']**2 / 2. / 1.25663706212*10**(-6) / 10**9    * 10**6 *10**9 #nT to T # J/cm^3 to nPa
    df['P'] = df['P_t'] + df['P_B']
    df['Beta'] = df['P_t'] / df['P_B']
    df['POL'] = np.sign(df['B_R'] - df['B_T']*df['R']*2.7*10**(-6)/df['V'])
    df['S_P'] = df['T']/df['N']**(2./3.)/11604.5
    
    df[df['V']>1500] = df[df['V']>1500]*np.nan
    df[df['B']>500] = df[df['B']>500]*np.nan
    df.to_parquet('reduced_data\solo\solo_data'+month+'.parquet')

    solo.plot(df)
    plt.show()



In [ ]:
from CIRESA import stereo_a, stereo_a, stereo_a, stereo_a, stereo_a
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define the start and end dates
start_date = '2020-01'
#start_date = '2019-11'
end_date = '2024-09'

# Generate a date range with monthly frequency
months = pd.date_range(start=start_date, end=end_date, freq='M').strftime('%Y-%m').tolist()

for month in months:
    try:
        df = stereo_a.load(month=month)
        df['Spacecraft_ID'] = 4
        #print(df)
        df.to_parquet('reduced_data\stereo_a\stereo_a_data'+month+'.parquet')
    except Exception as e:
        print(f"Error loading data for month {month}: {e}")


